In [10]:
import networkx as nx
import numpy as np
import random
import scipy.sparse as sp
from joblib import Parallel, delayed
from scipy.stats import norm
from pyrwr.rwr import RWR

def save_graph_to_file(G, file_path):
    """Save the graph as an edge list file."""
    with open(file_path, 'w') as f:
        for u, v in G.edges():
            f.write(f"{u}\t{v}\t1.0\n")

def get_connected_components(adj_matrix):
    """Find connected components using sparse matrix operations."""
    _, labels = sp.csgraph.connected_components(adj_matrix, directed=False)
    return labels

def compute_subnetwork_scores(adj_matrix, scores):
    """Compute subnetwork scores using connected components."""
    labels = get_connected_components(adj_matrix)
    unique_labels, sums = np.unique(labels, return_counts=False), np.zeros(labels.max() + 1)

    np.add.at(sums, labels, scores)
    return sums, labels

def permuted_max_score(adj_matrix, scores):
    """Shuffle scores and return the max subnetwork score."""
    random.shuffle(scores)
    return max(compute_subnetwork_scores(adj_matrix, scores)[0])

def generate_null_distribution(adj_matrix, scores, n_permutations=1000, n_jobs=-1):
    """Parallelized null distribution computation."""
    return Parallel(n_jobs=n_jobs)(
        delayed(permuted_max_score)(adj_matrix, scores.copy()) for _ in range(n_permutations)
    )

def compute_p_values(actual_scores, null_distribution):
    """Compute p-values by comparing actual subnetwork scores to the null distribution."""
    null_distribution = np.array(null_distribution)
    p_values = [(np.sum(null_distribution >= score) + 1) / (len(null_distribution) + 1) for score in actual_scores]
    return np.array(p_values)

def compute_rwr_scores(graph_file, seed_scores, restart_prob=0.85):
    """Compute RWR-based node scores using pyrwr.RWR."""
    rwr = RWR()
    rwr.read_graph(graph_file, graph_type="undirected")
    
    # Use only one seed node as per pyrwr's RWR function
    seed_node = max(seed_scores, key=seed_scores.get)  # Select the highest-weight seed node
    rwr_scores = rwr.compute(seed_node, c=restart_prob)
    return np.array(rwr_scores)

def find_significant_subnetworks(graph_file, seed_scores, adj_matrix, nodes, alpha=0.05, n_permutations=1000):
    """Find significant subnetworks using RWR-based scores."""
    scores = compute_rwr_scores(graph_file, seed_scores)
    actual_scores, labels = compute_subnetwork_scores(adj_matrix, scores)
    null_distribution = generate_null_distribution(adj_matrix, scores, n_permutations)
    p_values = compute_p_values(actual_scores, null_distribution)
    threshold = np.percentile(null_distribution, 100 * (1 - alpha))

    significant_subnetworks = []
    for i, (score, p_val) in enumerate(zip(actual_scores, p_values)):
        if score > threshold:
            significant_nodes = [nodes[j] for j in range(len(nodes)) if labels[j] == i]
            significant_subnetworks.append({"nodes": significant_nodes, "score": score, "p_value": p_val})

    return significant_subnetworks, threshold

# Example Usage
graph_file = "graph_edges.tsv"
G = nx.erdos_renyi_graph(5000, 0.001)
save_graph_to_file(G, graph_file)

nodes = list(G.nodes())
node_index = {node: i for i, node in enumerate(nodes)}

deg_matrix = sp.diags([G.degree(n) for n in nodes])
adj_matrix = nx.to_scipy_sparse_array(G, nodelist=nodes, format='csr')

seed_scores = {random.choice(nodes): random.uniform(0, 1) for _ in range(10)}

significant_subnetworks, threshold = find_significant_subnetworks(graph_file, seed_scores, adj_matrix, nodes)
print(f"Found {len(significant_subnetworks)} significant subnetworks.")
for sub in significant_subnetworks[:5]:
    print(f"Subnetwork with {len(sub['nodes'])} nodes | Score: {sub['score']:.2f} | p-value: {sub['p_value']:.4f}")

print(f"Threshold Score: {threshold:.2f}")


The iteration has converged at 7-iter:   7%|▋         | 7/100 [00:00<00:00, 163.37it/s]


Found 1 significant subnetworks.
Subnetwork with 4980 nodes | Score: 1.00 | p-value: 0.0010
Threshold Score: 1.00


In [13]:
import networkx as nx
import numpy as np
import random
import scipy.sparse as sp
from joblib import Parallel, delayed
from statsmodels.stats.multitest import multipletests
from pyrwr.rwr import RWR

def save_graph_to_file(G, file_path):
    """Save the graph as an edge list file."""
    with open(file_path, 'w') as f:
        for u, v in G.edges():
            f.write(f"{u}\t{v}\t1.0\n")

def get_connected_components(adj_matrix):
    """Find connected components using sparse matrix operations."""
    _, labels = sp.csgraph.connected_components(adj_matrix, directed=False)
    return labels

def compute_subnetwork_scores(adj_matrix, scores):
    """Compute subnetwork scores using connected components."""
    labels = get_connected_components(adj_matrix)
    unique_labels, sums = np.unique(labels, return_counts=False), np.zeros(labels.max() + 1)

    np.add.at(sums, labels, scores)
    return sums, labels

def permuted_max_score(adj_matrix, scores):
    """Shuffle scores and return the max subnetwork score."""
    random.shuffle(scores)
    return max(compute_subnetwork_scores(adj_matrix, scores)[0])

def generate_null_distribution(adj_matrix, scores, n_permutations=1000, n_jobs=-1):
    """Parallelized null distribution computation."""
    return Parallel(n_jobs=n_jobs)(
        delayed(permuted_max_score)(adj_matrix, scores.copy()) for _ in range(n_permutations)
    )

def compute_p_values(actual_scores, null_distribution):
    """Compute p-values and apply multiple testing correction."""
    null_distribution = np.array(null_distribution)
    p_values = [(np.sum(null_distribution >= score) + 1) / (len(null_distribution) + 1) for score in actual_scores]
    p_values = np.array(p_values)
    corrected_p_values = multipletests(p_values, method='fdr_bh')[1]  # Benjamini-Hochberg correction
    return corrected_p_values

def compute_rwr_scores(graph_file, seed_scores, restart_prob=0.85):
    """Compute RWR-based node scores using pyrwr.RWR."""
    rwr = RWR()
    rwr.read_graph(graph_file, graph_type="undirected")
    
    # Use only one seed node as per pyrwr's RWR function
    seed_node = max(seed_scores, key=seed_scores.get)  # Select the highest-weight seed node
    rwr_scores = rwr.compute(seed_node, c=restart_prob)
    return np.array(rwr_scores)

def find_significant_subnetworks(graph_file, seed_scores, adj_matrix, nodes, alpha=0.05, n_permutations=1000):
    """Find significant subnetworks using RWR-based scores."""
    scores = compute_rwr_scores(graph_file, seed_scores)
    actual_scores, labels = compute_subnetwork_scores(adj_matrix, scores)
    null_distribution = generate_null_distribution(adj_matrix, scores, n_permutations)
    p_values = compute_p_values(actual_scores, null_distribution)
    threshold = np.percentile(null_distribution, 100 * (1 - alpha))

    significant_subnetworks = []
    for i, (score, p_val) in enumerate(zip(actual_scores, p_values)):
        if p_val < alpha:
            significant_nodes = [nodes[j] for j in range(len(nodes)) if labels[j] == i]
            significant_subnetworks.append({"nodes": significant_nodes, "score": score, "p_value": p_val})

    return significant_subnetworks, threshold

# Example Usage
graph_file = "graph_edges.tsv"
G = nx.erdos_renyi_graph(5000, 0.001)
save_graph_to_file(G, graph_file)

nodes = list(G.nodes())
node_index = {node: i for i, node in enumerate(nodes)}

deg_matrix = sp.diags([G.degree(n) for n in nodes])
adj_matrix = nx.to_scipy_sparse_array(G, nodelist=nodes, format='csr')

seed_scores = {random.choice(nodes): random.uniform(0, 1) for _ in range(10)}

significant_subnetworks, threshold = find_significant_subnetworks(graph_file, seed_scores, adj_matrix, nodes)
print(f"Found {len(significant_subnetworks)} significant subnetworks.")
for sub in significant_subnetworks[:5]:
    print(f"Subnetwork with {len(sub['nodes'])} nodes | Score: {sub['score']:.2f} | p-value: {sub['p_value']:.4f}")

print(f"Threshold Score: {threshold:.2f}")


The iteration has converged at 7-iter:   7%|▋         | 7/100 [00:00<00:00, 760.55it/s]


Found 1 significant subnetworks.
Subnetwork with 4970 nodes | Score: 1.00 | p-value: 0.0290
Threshold Score: 1.00
